In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
# import tokenizer library for NLP
from pyspark.ml.feature import Tokenizer

In [6]:
# create DF
dataframe = spark.createDataFrame([
                                   (0, 'Spark is great.'),
                                   (1, 'We are learning Spark.'),
                                   (2, 'Spark is better than Hadoop no doibt')
], ['id', 'sentence'])
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|     Spark is great.|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [7]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_1cb677131f4f

In [9]:
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)

+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great.                     |[spark, is, great.]                         |
|1  |We are learning Spark.              |[we, are, learning, spark.]                 |
|2  |Spark is better than Hadoop no doibt|[spark, is, better, than, hadoop, no, doibt]|
+---+------------------------------------+--------------------------------------------+



In [11]:
# make user define function (UDF) that takes list of words in and return list length
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [12]:
# Next, we'll import the udf function, the col function to select a column to be passed into a function, 
# and the type IntegerType that will be used in our udf to define the data type of the output
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [13]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [15]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
# transform DF
tokenized_df = tokenizer.transform(dataframe)
# select needed columns and don't truncate results
tokenized_df.withColumn('tokens', count_tokens(col('words'))).show(truncate= False)

+---+------------------------------------+--------------------------------------------+------+
|id |sentence                            |words                                       |tokens|
+---+------------------------------------+--------------------------------------------+------+
|0  |Spark is great.                     |[spark, is, great.]                         |3     |
|1  |We are learning Spark.              |[we, are, learning, spark.]                 |4     |
|2  |Spark is better than Hadoop no doibt|[spark, is, better, than, hadoop, no, doibt]|7     |
+---+------------------------------------+--------------------------------------------+------+

